In [ ]:
%pylab inline
%matplotlib inline

import os
import sys
import numpy as np
import pickle

import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [ ]:
current_path = !pwd
parent_path = os.path.dirname(current_path[0])
if parent_path not in sys.path: sys.path.append(parent_path)
work_path = "/".join(parent_path.split('/')[:-1])
if work_path not in sys.path: sys.path.append(work_path)
    
import DeepSparseCoding as dsc
from DeepSparseCoding.data.dataset import Dataset
from DeepSparseCoding.analysis import analysis_picker as ap
from DeepSparseCoding.utils import data_processing as dp
import utils.model_handling as model_funcs
import utils.dataset_generation as iso_data
import utils.histogram_analysis as hist_funcs
import utils.plotting as pf

In [ ]:
text_width = 416.83269 #pt = 14.65cm
fontsize = 12
dpi = 800
font_settings = {
        "text.usetex": True,
        "font.family": "serif",
        "axes.labelsize": fontsize,
        "axes.titlesize": fontsize,
        "figure.titlesize": fontsize,
        "font.size": fontsize,
        "legend.fontsize": fontsize,
        "xtick.labelsize": fontsize-2,
        "ytick.labelsize": fontsize-2,
}
mpl.rcParams.update(font_settings)
mpl.pyplot.rc('text', usetex=True)

color_vals = dict(zip(["blk", "lt_green", "md_green", "dk_green", "lt_blue", "md_blue", "dk_blue", "lt_red", "md_red", "dk_red"],
  ["#000000", "#A9DFBF", "#196F3D", "#27AE60", "#AED6F1", "#3498DB", "#21618C", "#F5B7B1", "#E74C3C", "#943126"]))

In [ ]:
def get_dsc_activations_cell(analyzer, images, neuron, batch_size=10, activation_operation=None):
    """
    Returns the activations from a model for given input images
    Parameters:
        analyzer [DSC analyzer object] an object from the DeepSparseCoding library
        images [np.ndarray] of size NumImages x W x H
        neuron [int or vector of ints] that points to the neuron index
        batch_size [int] specifying the batch size to use for the getting the neuron activations
        activation_operation [function] to be used if the DSC model has a unique function handle for getting neuron activations (e.g. in the case of lca_subspace)
    Output:
        activations [np.ndarray] vector of length len(neuron)
    """
    images = dp.reshape_data(images[..., None], flatten=analyzer.model.params.vectorize_data)[0]
    activations = analyzer.compute_activations(images, batch_size, activation_operation)[:, neuron]
    return activations

def load_analysis(params):
    params.model_dir = (work_path+"/Projects/"+params.model_name)
    analyzer = ap.get_analyzer(params.model_type)
    analyzer.setup(params)
    analyzer.model.setup(analyzer.model_params)
    analyzer.load_analysis(save_info=params.save_info)
    return params, analyzer

## Pre-trained LCA model

In [ ]:
class lca_1024_vh_params(object):
  def __init__(self):
    self.model_type = "lca"
    self.model_name = "lca_1024_vh"
    self.display_name = "Sparse Coding"
    self.version = "0.0"
    self.save_info = "analysis_train_carlini_targeted"
    self.overwrite_analysis_log = False
    self.use_group_activations = False

In [ ]:
params, analyzer = load_analysis(lca_1024_vh_params())
neuron_weights = [analyzer.bf_stats["basis_functions"][idx]
    for idx in range(len(analyzer.bf_stats["basis_functions"]))]

In [ ]:
cont_analysis = dict()

#cont_analysis["target_neuron_ids"] = get_rand_target_neuron_ids(3, len(meis["images"]))
cont_analysis["target_neuron_ids"] = [992, 381, 62]# [1, 2, 3]
cont_analysis["num_comparisons"] = 3
cont_analysis["min_angle"] = 5
cont_analysis["x_range"] = (-2.0, 2.0)
cont_analysis["y_range"] = (-2.0, 2.0)
cont_analysis["num_images"] = int(30**2)
cont_analysis["image_scale"] = 33 # norm of training data

iso_vectors = iso_data.compute_rand_vectors(
    [neuron_weights[idx] for idx in cont_analysis["target_neuron_ids"]],
    cont_analysis["num_comparisons"])
cont_analysis["rand_target_vectors"] = iso_vectors[0]
cont_analysis["rand_orth_vectors"] = iso_vectors[1]

iso_vectors = iso_data.compute_comp_vectors(
    neuron_weights,
    cont_analysis["target_neuron_ids"],
    cont_analysis["min_angle"],
    cont_analysis["num_comparisons"])
cont_analysis["comparison_neuron_ids"] = iso_vectors[0]
cont_analysis["comparison_target_vectors"] = iso_vectors[1]
cont_analysis["comparison_vectors"] = iso_vectors[2]

cont_analysis["target_vectors"] = cont_analysis["comparison_target_vectors"]

cont_analysis["contour_dataset"], datapoints = iso_data.get_contour_dataset(
    cont_analysis["target_vectors"],
    cont_analysis["comparison_vectors"],
    cont_analysis["x_range"],
    cont_analysis["y_range"],
    cont_analysis["num_images"],
    cont_analysis["image_scale"])

activation_function_kwargs = {
    "batch_size": 100}

cont_analysis["activations"] = model_funcs.get_normalized_activations(
    analyzer,
    cont_analysis["target_neuron_ids"],
    datapoints,
    get_dsc_activations_cell,
    activation_function_kwargs)

In [ ]:
width_fraction = 1.0
show_contours = True
num_levels = 10
contour_fig, contour_handles = pf.plot_group_iso_contours(cont_analysis, num_levels, show_contours, text_width, width_fraction, dpi)

## Pre-trained Subspace-LCA model

In [ ]:
class lca_subspace_vh_params(object):
  def __init__(self):
    self.model_type = "lca_subspace"
    self.model_name = "lca_subspace_vh"
    self.display_name = "Subspace Sparse Coding"
    self.version = "5x_4_1.0_0.2"
    self.save_info = "analysis_train"
    self.overwrite_analysis_log = False
    self.use_group_activations = False

In [ ]:
params, analyzer = load_analysis(lca_subspace_vh_params())
neuron_weights = [analyzer.bf_stats["basis_functions"][idx]
    for idx in range(len(analyzer.bf_stats["basis_functions"]))]

In [ ]:
"""
The following is a somewhat arbitrary way of choosing some target & comparison vectors that should exhibit both exo- and endo-origin curvature
"""

num_target = 3
num_within_group = 1
num_outside_group = 2
min_angle = 15


def unique(item_list):
    seen = set()
    unique_list = []
    for x in item_list:
        if x not in seen:
            unique_list.append(x)
            seen.add(x)
    return unique_list


angle_matrix = iso_data.get_vector_angles(neuron_weights)[1]
num_below_min = np.count_nonzero(angle_matrix<min_angle) # many angles are -1 or 0
sorted_angle_indices = np.stack(np.unravel_index(np.argsort(angle_matrix.ravel()),
    angle_matrix.shape), axis=1)[num_below_min:, :]

unique_target_indices = unique(sorted_angle_indices[:,0]) # sorted list of unique target indices

all_target_group_assignments = [
    analyzer.model.module.group_assignments[index]
    for index in unique_target_indices] # list of groups for all possible target indices

unique_target_group_assignments = unique(all_target_group_assignments) # sorted list of unique groups
target_groups = unique_target_group_assignments[:num_target]

selected_target_indices = [analyzer.model.module.group_ids[group_number][0]
   for group_number in target_groups] # first neuron in each group

within_group_indices = [list(analyzer.model.module.group_ids[group_number][1:num_within_group+1])
    for group_number in target_groups] # alternate neuron within the same group as target neuron

outside_group_indices = []
for group_index, group_number in enumerate(target_groups):
    sub_list = []
    # loop through a sorted list of all possible comparison IDs, pick the first ones that are in a different group
    for candidate_comp_index in sorted_angle_indices[selected_target_indices[group_index], :]:
        if analyzer.model.module.group_assignments[candidate_comp_index] != group_number:
            sub_list.append(candidate_comp_index)
    outside_group_indices.append(sub_list[:num_outside_group])

selected_comparison_indices = []
selected_comparison_indices = [within_group_indices[group_index]+outside_group_indices[group_index]
    for group_index in range(len(target_groups))]

In [ ]:
cont_analysis = dict()

cont_analysis["target_neuron_ids"] = selected_target_indices
cont_analysis["comparison_neuron_ids"] = selected_comparison_indices
cont_analysis["num_comparisons"] = num_within_group+num_outside_group
cont_analysis["min_angle"] = min_angle
cont_analysis["x_range"] = (-2.0, 2.0)
cont_analysis["y_range"] = (-2.0, 2.0)
cont_analysis["num_images"] = int(30**2)
cont_analysis["image_scale"] = 33 # norm of training data


iso_vectors = iso_data.compute_specified_vectors(
    neuron_weights,
    cont_analysis["target_neuron_ids"],
    cont_analysis["comparison_neuron_ids"])
cont_analysis["comparison_target_vectors"] = iso_vectors[0]
cont_analysis["comparison_vectors"] = iso_vectors[1]

cont_analysis["target_vectors"] = cont_analysis["comparison_target_vectors"]

cont_analysis["contour_dataset"], datapoints = iso_data.get_contour_dataset(
    cont_analysis["target_vectors"],
    cont_analysis["comparison_vectors"],
    cont_analysis["x_range"],
    cont_analysis["y_range"],
    cont_analysis["num_images"],
    cont_analysis["image_scale"])

activation_function_kwargs = {
    "batch_size": 100,
    "activation_operation": analyzer.model.get_reshaped_group_activity}

cont_analysis["activations"] = model_funcs.get_normalized_activations(
    analyzer,
    cont_analysis["target_neuron_ids"],
    datapoints,
    get_dsc_activations_cell,
    activation_function_kwargs)

In [ ]:
width_fraction = 1.0
show_contours = True
num_levels = 10
contour_fig, contour_handles = pf.plot_group_iso_contours(cont_analysis, num_levels, show_contours, text_width, width_fraction, dpi)

## Lambda model

In [ ]:
input_shape = [16, 16]

lamb_activation = lambda x : tf.identity(x) # linear
#lamb_activation = lambda x : tf.nn.relu(0, x) # ReLU
#lamb_activation = lambda x : tf.math.sigmoid(x) # Sigmoid
#lamb_activation = lambda x : x / tf.reduce_sum(tf.square(x), axis=1, keepdims=True) # div norm

## TODO: energy model defined this way will only have a single output - so the current code can't handle it
### We will want to set up something like what is done in lca_subspace.get_reshaped_group_activity
#lamb_activation = lambda x : tf.reduce_sum(tf.square(x), axis=1, keepdims=True) # energy model

neuron_weights = np.random.normal(loc=0.0, scale=1.0, size=[np.prod(input_shape), np.prod(input_shape)])

lambda_params = dsc.params.param_picker.get_params("lambda")
lambda_params.set_data_params("synthetic")
lambda_params.batch_size = 1 # TODO: support batches in get_lambda_activations_cell
lambda_params.data_shape = [np.prod(input_shape)] # assumes vector inputs (i.e. not convoultional)
lambda_params.activation_function = lamb_activation

lambda_model = dsc.models.model_picker.get_model("lambda")
lambda_model.setup(lambda_params)

In [ ]:
def get_lambda_activations_cell(model, images, neuron, weights):
    """
    Returns the activations from a model for given input images
    Parameters:
        analyzer [DSC analyzer object] an object from the DeepSparseCoding library
        images [np.ndarray] of size NumImages x W x H
        neuron [int or vector of ints] that points to the neuron index
    Output:
        activations [np.ndarray] vector of length len(neuron)
    """
    num_images, width, height = images.shape
    images = images.reshape([num_images, width*height]) # vectorize images
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    with tf.compat.v1.Session(config=config, graph=model.graph) as sess:
        feed_dict = model.get_feed_dict(images[0, ...][None, ...])
        sess.run(model.init_op, feed_dict)
        sess.graph.finalize()
        activations = []
        for img_idx in range(num_images):
            image = images[img_idx, ...]
            feed_dict = model.get_feed_dict(image[None, ...])
            feed_dict[model.weight_placeholder] = weights
            all_activations = sess.run(model.get_encodings(), feed_dict)
            activations.append(all_activations[:, neuron])
    return np.stack(activations, axis=0)

In [ ]:
cont_analysis = dict()

cont_analysis["target_neuron_ids"] = [0, 1, 2]
cont_analysis["num_comparisons"] = 3
cont_analysis["min_angle"] = 5
cont_analysis["x_range"] = (-2.0, 2.0)
cont_analysis["y_range"] = (-2.0, 2.0)
cont_analysis["num_images"] = int(30**2)
cont_analysis["image_scale"] = 1 # norm of training data

iso_vectors = iso_data.compute_comp_vectors(
    [neuron_weights[:, weight_idx].reshape(input_shape) for weight_idx in range(np.prod(input_shape))],
    cont_analysis["target_neuron_ids"],
    cont_analysis["min_angle"],
    cont_analysis["num_comparisons"])
cont_analysis["comparison_neuron_ids"] = iso_vectors[0]
cont_analysis["comparison_target_vectors"] = iso_vectors[1]
cont_analysis["comparison_vectors"] = iso_vectors[2]

cont_analysis["target_vectors"] = cont_analysis["comparison_target_vectors"]

cont_analysis["contour_dataset"], datapoints = iso_data.get_contour_dataset(
    cont_analysis["target_vectors"],
    cont_analysis["comparison_vectors"],
    cont_analysis["x_range"],
    cont_analysis["y_range"],
    cont_analysis["num_images"],
    cont_analysis["image_scale"])

activation_function_kwargs = {"weights": neuron_weights}

cont_analysis["activations"] = model_funcs.get_normalized_activations(
    lambda_model,
    cont_analysis["target_neuron_ids"],
    datapoints,
    get_lambda_activations_cell,
    activation_function_kwargs)

In [ ]:
width_fraction = 1.0
show_contours = True
num_levels = 10
contour_fig, contour_handles = pf.plot_group_iso_contours(cont_analysis, num_levels, show_contours, text_width, width_fraction, dpi)